In [3]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# 📦 Install Google Chrome
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb || apt-get -fy install > /dev/null

# 📥 Get matching ChromeDriver
import os, re, requests, zipfile, io

# Get Chrome version
version_output = !google-chrome --version
major_version = re.search(r'(\d+)\.', version_output[0]).group(1)

# Get matching ChromeDriver URL
resp = requests.get('https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json')
driver_info = resp.json()['channels']['Stable']['downloads']['chromedriver']
driver_url = next(d['url'] for d in driver_info if d['platform'] == 'linux64')

# Download and extract ChromeDriver
driver_zip = requests.get(driver_url)
z = zipfile.ZipFile(io.BytesIO(driver_zip.content))
extract_path = "/usr/bin/chromedriver-linux64"
z.extractall(extract_path)
os.chmod(f"{extract_path}/chromedriver", 0o755)

# 🔄 Fix symlink safely
if os.path.islink("/usr/bin/chromedriver") or os.path.exists("/usr/bin/chromedriver"):
    os.remove("/usr/bin/chromedriver")
os.symlink(f"{extract_path}/chromedriver", "/usr/bin/chromedriver")

# ✅ Install Python packages
!pip install -q selenium beautifulsoup4

# ✅ Test it works
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)
driver.get("https://example.com")
print("✅ Success! Page title:", driver.title)
driver.quit()

(Reading database ... 126897 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (138.0.7204.183-1) over (138.0.7204.183-1) ...
Setting up google-chrome-stable (138.0.7204.183-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
✅ Success! Page title: Example Domain


In [4]:
from google.colab import userdata
API_KEY = userdata.get('NY_TIMES_KEY')

In [6]:
def send_request(date):
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    try:
        response = requests.get(url, verify=False).json()
    except Exception:
        return None
    time.sleep(6)
    return response


def is_valid(article, date):
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    data = {'headline': [],
        'date': [],
        'web_url': [],
        'doc_type': [],
        'lead_paragraph': [],
        'material_type': [],
        'author': [],
        'section': [],
        'subsection': [],
        'keywords': []}

    articles = response['response']['docs']
    for article in articles:
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main'])
            if 'section_name' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            if 'lead_paragraph' in article:
                data['lead_paragraph'].append(article['lead_paragraph'])
            else:
                data['lead_paragraph'].append(None)
            if 'web_url' in article:
                data['web_url'].append(article['web_url'])
            else:
                data['web_url'].append(None)
            if 'subsection_name' in article:
                data['subsection'].append(article['subsection_name'])
            else:
                data['subsection'].append(None)
            if 'byline' in article:
                data['author'].append(article['byline']['original'])
            else:
                data['author'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article:
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data)


def get_data(dates):
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        print('Working on ' + str(date) + '...')
        csv_path = 'headlines/' + date[0] + '-' + date[1] + '.csv'
        if not os.path.exists(csv_path): # If we don't already have this month
            response = send_request(date)
            if response is not None:
                df = parse_response(response)
                total += len(df)
                df.to_csv(csv_path, index=False)
                print('Saving ' + csv_path + '...')
    print('Number of articles collected: ' + str(total))

In [15]:
start = datetime.date(2025, 1, 1)
end = datetime.date(2025, 2, 15)

In [16]:
months = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]

In [17]:
get_data(months)

Date range: ['2025', '1'] to ['2025', '2']
Working on ['2025', '1']...
Saving headlines/2025-1.csv...
Working on ['2025', '2']...
Saving headlines/2025-2.csv...
Number of articles collected: 6088


In [18]:
import os
import glob
import pandas as pd
os.chdir("/content/headlines") ## use Google Colab

In [19]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [20]:
#combine in a single file
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [21]:
def get_body(url):
    driver.get(url)
    article_text = ''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    paragraph = soup.find_all('p')
    for i in paragraph:
        a = i.get_text()
        if a != 'Advertisement' and a != 'Supported by' and a != 'Send any friend a story' and a != 'As a subscriber, you have 10 gift articles to give each month. Anyone can read what you share.' and not a.startswith("By"):
            article_text += a
            article_text += " "
    time.sleep(8)
    return article_text

In [22]:
df = pd.read_csv("combined_csv.csv")

In [ ]:
## CREATE A NEW COLUMN IN THE DATAFRAME CALLED "article_doby" WHERE WE ARE GOING TO APPEND THE TEXT RETRIEVE FROM
## THE URL. NOTE: web_url REFERS TO THE COLUMN IN THE DATASET CONTAINING THE URL OF THE ARTICLE TO PASS TO get_body.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)
df['article_body'] = df.head(100).apply(lambda x: get_body(x.web_url), axis=1)

In [27]:
for article in df['article_body'].dropna():
    print(article)

We and our vendors use cookies and similar methods (“Cookies”) to recognize visitors and remember their preferences. We also use Cookies for a variety of purposes, including analytics, to measure marketing effectiveness and to target and measure the effectiveness of ads. You can accept or reject the use of Cookies for individual purposes below. Some vendors process your data on the basis of their legitimate interest - you can object to such processing below. Your preferences will be saved in a cookie named “fides_consent” for a maximum duration of 12 months, as well as in your registered user account if you are logged in. If you previously accepted these methods through our prior banner, then we will use your data for targeting. Your preferences will apply on nytimes.com, as well as our News, Cooking, Games and Audio apps. Your preferences here are unrelated to Apple’s App Tracking Transparency Framework. THEATER REVIEW The paint on the balconies of the Majestic Theater looks chipped a